In [1]:
from rdflib import Graph, Namespace, URIRef, Literal, BNode
from rdflib.namespace import RDF, RDFS, OWL
from owlrl import DeductiveClosure, OWLRL_Semantics
from intension import Intension
from fn_example_generator import FNExampleGenerator
from tqdm import tqdm
import json, os, random

In [2]:
# Get Turtle serialization of vrd_world_v1.owl
VRD_WORLD_OWL = open("data/NeSy4VRD/nesy4vrd_ontology/vrd_world_v1.owl", "r").read()

In [3]:
training_results = json.load(open('experiments/nesy4vrd/claude-3-haiku-20240307-train.json', 'r'))

In [4]:
fns = [ result for result in training_results if result['answer'] == '0' ]

In [5]:
len(fns)

29

In [6]:
for result in fns:
    result['graph'] = VRD_WORLD_OWL

In [7]:
generator = FNExampleGenerator("claude-3-haiku-20240307")

/Users/bradleyallen/Documents/GitHub/predictable-logical-inference-with-llm-intensions/env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [12]:
revisions = [ generator.chain.invoke(fn) for fn in tqdm(fns) ]

100%|██████████| 29/29 [01:36<00:00,  3.32s/it]


In [14]:
for revision in revisions:
    revision['answer'] = '1'
    revision['rationale'] = revision['text']['revision']
    revision.pop('text')
    revision.pop('model')
    revision.pop('graph')

In [16]:
revisions[0]

{'s': 'vrd:skateOn',
 'p': 'rdfs:domain',
 'o': 'vrd:PullCapableThing',
 'rationale': 'The given knowledge graph triple is true based on the ontology information provided.\n\nThe ontology states that the property \'vrd:skateOn\' has the following definition:\n\nvrd:skateOn rdf:type owl:ObjectProperty ;\n            rdfs:subPropertyOf vrd:on ;\n            rdfs:domain vrd:Person ;\n            rdfs:range vrd:SkateboardingRelatedThing ;\n            rdfs:comment "Property \'skateOn\' has a consistent meaning/usage relating to skateboarding."@en .\n\nThis indicates that the domain of the \'vrd:skateOn\' property is the class \'vrd:Person\'. The ontology also defines the class \'vrd:PullCapableThing\' as:\n\nvrd:PullCapableThing rdf:type owl:Class ;\n                     owl:equivalentClass [ rdf:type owl:Class ;\n                                           owl:unionOf ( vrd:Horse\n                                                         vrd:Person\n                                         

In [22]:
def exemplar_str(ex):
    return f"""###
Subject: <{ex["s"]}>
Predicate: <{ex["p"]}>
Object: <{ex["o"]}>
Rationale: {ex["rationale"]}
Answer: {ex["answer"]}
"""

In [24]:
print(exemplar_str(revisions[0]))

###
Subject: <vrd:skateOn>
Predicate: <rdfs:domain>
Object: <vrd:PullCapableThing>
Rationale: The given knowledge graph triple is true based on the ontology information provided.

The ontology states that the property 'vrd:skateOn' has the following definition:

vrd:skateOn rdf:type owl:ObjectProperty ;
            rdfs:subPropertyOf vrd:on ;
            rdfs:domain vrd:Person ;
            rdfs:range vrd:SkateboardingRelatedThing ;
            rdfs:comment "Property 'skateOn' has a consistent meaning/usage relating to skateboarding."@en .

This indicates that the domain of the 'vrd:skateOn' property is the class 'vrd:Person'. The ontology also defines the class 'vrd:PullCapableThing' as:

vrd:PullCapableThing rdf:type owl:Class ;
                     owl:equivalentClass [ rdf:type owl:Class ;
                                           owl:unionOf ( vrd:Horse
                                                         vrd:Person
                                                         vrd

In [25]:
exemplars = "".join([ exemplar_str(ex) for ex in revisions ])

In [30]:
print(exemplars, file=open('examples.json', 'w+'))

In [29]:
json.dump(exemplars, open("examples.json", "w+"))